In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
print(tf.__version__)

In [ ]:
train_dir = '/kaggle/input/facecare-dataset-2-1/train'
test_dir = '/kaggle/input/facecare-dataset-2-1/valid'
img_height, img_width = 600, 600
batch_size = 32

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.2, 1),
    horizontal_flip=True,
    fill_mode='nearest'
)

# Normalization
test_datagen = ImageDataGenerator(rescale=1./255)

# Load and iterate training dataset
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
)

# Load and iterate validation dataset
validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
)
# Calculate steps_per_epoch and validation_steps
num_train_samples = len(train_generator.filenames)
num_validation_samples = len(validation_generator.filenames)

steps_per_epoch = math.ceil(num_train_samples / batch_size)
validation_steps = math.ceil(num_validation_samples / batch_size)

In [ ]:
from sklearn.utils import class_weight

train_labels = train_generator.classes

# Compute class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)

# Convert the class weights to a dictionary
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Print the class weights
print("Class weights:", class_weights_dict)

In [ ]:
from tensorflow.keras import regularizers

base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers
# for layer in base_model.layers:
#     layer.trainable = False
base_model.trainable = False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.001)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/best_faceCare_model_Inception.keras',  # Path to save the model
    monitor='val_accuracy',  # Monitor validation accuracy 
    save_best_only=True,  # Only save the best model
    mode='max'  # Save when validation accuracy reaches its maximum
)

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5),
              metrics=['accuracy'])
# Model summary
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs = 100,
    validation_data= validation_generator,
    class_weight=class_weights_dict,
)

In [ ]:
def plot_loss_acc(history):
  '''Plots the training and validation loss and accuracy from a history object'''
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'bo', label='Training accuracy')
  plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'bo', label='Training Loss')
  plt.plot(epochs, val_loss, 'b', label='Validation Loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

In [ ]:
plot_loss_acc(history)

In [ ]:
model.save("/kaggle/working/FaceCare_Xception_v1.keras")
model.save("/kaggle/working/FaceCare_Xception_v1.h5")

In [ ]:
import random
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
import cv2
from PIL import Image

def load_and_preprocess_image(img_path, img_height, img_width):
    img = image.load_img(img_path, target_size=(img_height, img_width))  # Adjust target size as needed
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize pixel values
    return img_array

def compute_saliency(model, image):
    image = tf.convert_to_tensor(image)
    with tf.GradientTape() as tape:
        tape.watch(image)
        predictions = model(image)
        loss = predictions[:, 0]  # For binary classification, assuming the first class is the positive class
    gradients = tape.gradient(loss, image)
    saliency = tf.reduce_max(tf.abs(gradients), axis=-1)
    return saliency.numpy()
    
def show_saliency(img, img_height, img_width, model):
    
    test_image = load_and_preprocess_image(img, img_height, img_width)
    saliency_map = compute_saliency(model, test_image)
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title("Original Image")
    plt.imshow(test_image.squeeze(), cmap='gray')

    plt.subplot(1, 2, 2)
    plt.title("Saliency Map")
    plt.imshow(saliency_map.squeeze(), cmap='hot')

    plt.show()
    
def preprocess_image(img_path, target_size=(img_height, img_width)):
    img = Image.open(img_path)
    img = img.resize(target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize the image
    return img_array

def make_prediction(model, img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)[0][0]  # Get the predicted probability for the class
    if prediction > 0.5:
        return "No Acne", prediction
    else:
        return "Acne", prediction

In [ ]:
img1 = "/kaggle/input/facecare-dataset-2-1/predAcne.jpg"
img2 = "/kaggle/input/facecare-dataset-2-1/predAcne1.jpg"
img3 = "/kaggle/input/facecare-dataset-2-1/predNoAcne.png"

prediction1, probability1 = make_prediction(model, img1)
prediction2, probability2 = make_prediction(model, img2)
prediction3, probability3 = make_prediction(model, img3)

print(f"Prediction: {prediction1}")
print(f"Predicted Probability: {probability1}")

img = Image.open(img1)
plt.imshow(img)
plt.title(f"Prediction: {prediction1} ({probability1:.2f})")
plt.axis('off')  # Hide the axes
plt.show()

print(f"Prediction: {prediction2}")
print(f"Predicted Probability: {probability2}")

img = Image.open(img2)
plt.imshow(img)
plt.title(f"Prediction: {prediction2} ({probability2:.2f})")
plt.axis('off')  # Hide the axes
plt.show()

print(f"Prediction: {prediction2}")
print(f"Predicted Probability: {probability2}")

img = Image.open(img3)
plt.imshow(img)
plt.title(f"Prediction: {prediction3} ({probability3:.2f})")
plt.axis('off')  # Hide the axes
plt.show()

In [ ]:
show_saliency(img1, img_height, img_width, model)
show_saliency(img2, img_height, img_width, model)
show_saliency(img3, img_height, img_width, model)